In [1]:
"""Code to format info on movies in a csv file for publication."""

# libraries
import csv
import pandas as pd
import datetime

In [2]:
# Create list of dictionaries for each row.
listings = []
scrape_file = '2781_Fandango_data_Movie_full.csv'
with open(scrape_file, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        layout_dct = {'addy': row[0], 'theater': row[1], 'title': row[2], 'mpaa': row[3], 'stars': row[4], 'times': row[5], 'format': row[6], 'synopsis': row[7], 'date': row[8], 'city': row[9]}
        listings.append(layout_dct)
        
del listings[0] #remove header row   

In [4]:
# Create Pandas dataframe for listings
df = pd.DataFrame(listings)
times = (df.times.str.split('|', expand=True).rename(columns=lambda x: f"time_{x+1}"))
times = times.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
times = times.apply(lambda x: x.str.replace('p', ' p.m.') if x.dtype == "object" else x)
times = times.apply(lambda x: x.str.replace('a', ' a.m.') if x.dtype == "object" else x)
times = times.apply(lambda x: x.str.replace(':00', '') if x.dtype == "object" else x)
df = df.drop('times', axis=1)
df = pd.concat([df, times], axis=1, join_axes=[df.index])
df['streetaddy'] = df.addy.str.split(',').str[0]
df.streetaddy = df.streetaddy.str.replace('Blvd', 'Blvd.')
df.streetaddy = df.streetaddy.str.replace('Blvd..', 'Blvd.')
df.streetaddy = df.streetaddy.str.replace('Avenue', 'Ave')
df.streetaddy = df.streetaddy.str.replace('Ave', 'Ave.')
df.streetaddy = df.streetaddy.str.replace('Ave..', 'Ave.')
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['movieday'] = df['date'].dt.weekday_name
df = df.drop('city', axis=1)
df['cityaddy'] = df.addy.str.split(',').str[1]
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df = df.drop('addy', axis=1)
df

,date,format,mpaa,stars,synopsis,theater,title,time_1,time_2,time_3,time_4,time_5,time_6,time_7,time_8,streetaddy,movieday,cityaddy
0,2018-11-29,Standard,PG,4.5,Ralph and Vanellope embark on an adventure ins...,Ocala Center 6,Ralph Breaks the Internet,1 p.m.,4 p.m.,7 p.m.,None,None,None,None,None,2021 Silver Springs Blvd.,Thursday,Ocala
1,2018-11-29,Standard,PG-13,5,"Singer Freddie Mercury, guitarist Brian May, d...",Regal Royal Park Stadium 16,Bohemian Rhapsody,12:15 p.m.,3:20 p.m.,6:20 p.m.,9:30 p.m.,None,None,None,None,3702 Newberry Road,Thursday,Gainesville
2,2018-11-29,Standard,PG,4.5,Academy Award® nominee Benedict Cumberbatch le...,Regal Royal Park Stadium 16,Dr. Seuss' The Grinch (2018),12:05 p.m.,2:10 p.m.,4:25 p.m.,6:50 p.m.,9:10 p.m.,None,None,None,3702 Newberry Road,Thursday,Gainesville
3,2018-11-29,Standard,PG-13,5,"Singer Freddie Mercury, guitarist Brian May, d...",Regal Hollywood Stadium 16 & IMAX - Ocala,Bohemian Rhapsody,12:25 p.m.,3:55 p.m.,10:35 p.m.,None,None,None,None,None,2801 SW 27th Ave.,Thursday,Ocala
4,2018-11-29,Standard,PG-13,4.5,Reporter Eddie Brock develops superpowers afte...,Barnstorm Theater,Venom (2018),1 p.m.,4 p.m.,9:40 p.m.,None,None,None,None,None,2720 Brownwood Blvd.,Thursday,The Villages
5,2018-11-29,Standard,,5,,Regal Butler Town Center 14,2.0,12 p.m.,None,None,None,None,None,None,None,3101 SW 35th Blvd.,Thursday,Gainesville
6,2018-11-29,Standard,PG,4.5,Academy Award® nominee Benedict Cumberbatch le...,Celebration Pointe 10,Dr. Seuss' The Grinch (2018),1:15 p.m.,4:30 p.m.,7:10 p.m.,9:30 p.m.,None,None,None,None,4901 SW 31st Place,Thursday,Gainesville
7,2018-11-29,Standard,PG-13,4.5,In an effort to thwart Grindelwald's plans of ...,Celebration Pointe 10,Fantastic Beasts: The Crimes of Grindelwald,12 p.m.,3:15 p.m.,7 p.m.,1 p.m.,4:15 p.m.,7:30 p.m.,None,None,4901 SW 31st Place,Thursday,Gainesville
8,2018-11-29,Standard,PG,4.5,Academy Award® nominee Benedict Cumberbatch le...,Regal Butler Town Center 14,Dr. Seuss' The Grinch (2018),12:05 p.m.,2:30 p.m.,4:50 p.m.,10:45 p.m.,7:15 p.m.,None,None,None,3101 SW 35th Blvd.,Thursday,Gainesville
9,2018-11-29,Standard,PG,4.5,Ralph and Vanellope embark on an adventure ins...,Florida Twin,Ralph Breaks the Internet,7:30 p.m.,None,None,None,None,None,None,None,101 W. Call St.,Thursday,Starke


In [13]:
# Create print narrative for daily

pn = []
for index, row in df.iterrows():
    print('"',row['title'], '," ', row['theater'])

" Ralph Breaks the Internet ,"  Ocala Center 6
" Bohemian Rhapsody ,"  Regal Royal Park Stadium 16
" Dr. Seuss' The Grinch (2018) ,"  Regal Royal Park Stadium 16
" Bohemian Rhapsody ,"  Regal Hollywood Stadium 16 & IMAX - Ocala
" Venom (2018) ,"  Barnstorm Theater
" 2.0 ,"  Regal Butler Town Center 14
" Dr. Seuss' The Grinch (2018) ,"  Celebration Pointe 10
" Fantastic Beasts: The Crimes of Grindelwald ,"  Celebration Pointe 10
" Dr. Seuss' The Grinch (2018) ,"  Regal Butler Town Center 14
" Ralph Breaks the Internet ,"  Florida Twin
" Boy Erased ,"  Regal Hollywood Stadium 16 & IMAX - Ocala
" A Private War ,"  Old Mill Playhouse
" The Nutcracker and the Four Realms ,"  Regal Butler Town Center 14
" Fantastic Beasts: The Crimes of Grindelwald ,"  Regal Butler Town Center 14
" The Front Runner ,"  Barnstorm Theater
" The Nutcracker and the Four Realms ,"  Regal Royal Park Stadium 16
" Robin Hood (2018) ,"  Regal Royal Park Stadium 16
" The King And I: From The London Palladium ,"  Regal

In [11]:
print(pn)

['"', 'Ralph Breaks the Internet', '," ', 'Ocala Center 6', '\n', '"', 'Bohemian Rhapsody', '," ', 'Regal Royal Park Stadium 16', '\n', '"', "Dr. Seuss' The Grinch (2018)", '," ', 'Regal Royal Park Stadium 16', '\n', '"', 'Bohemian Rhapsody', '," ', 'Regal Hollywood Stadium 16 & IMAX - Ocala', '\n', '"', 'Venom (2018)', '," ', 'Barnstorm Theater', '\n', '"', '2.0', '," ', 'Regal Butler Town Center 14', '\n', '"', "Dr. Seuss' The Grinch (2018)", '," ', 'Celebration Pointe 10', '\n', '"', 'Fantastic Beasts: The Crimes of Grindelwald', '," ', 'Celebration Pointe 10', '\n', '"', "Dr. Seuss' The Grinch (2018)", '," ', 'Regal Butler Town Center 14', '\n', '"', 'Ralph Breaks the Internet', '," ', 'Florida Twin', '\n', '"', 'Boy Erased', '," ', 'Regal Hollywood Stadium 16 & IMAX - Ocala', '\n', '"', 'A Private War', '," ', 'Old Mill Playhouse', '\n', '"', 'The Nutcracker and the Four Realms', '," ', 'Regal Butler Town Center 14', '\n', '"', 'Fantastic Beasts: The Crimes of Grindelwald', '," ',

In [ ]:
def daily_pn(df):
    pn = []

In [3]:
# Convert datetime object into day of the week; But this is accomplished in the df work already.
def get_day(date_object=None):
    if not date_object:
        date_object = datetime.datetime.now()
    stamp = ""
    stamp += datetime.datetime.strftime(date_object, "%A")
    return(stamp)

In [ ]:
def make_listings(scrape_file):
    listings = []
    scrape_file = '2781_Fandango_data_Movie_full.csv'
    with open(scrape_file, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            layout_dct = {'addy': row[0], 'theater': row[1], 'title': row[2], 'mpaa': row[3], 'stars': row[4], 'times': row[5], 'format': row[6], 'synopsis': row[7], 'date': row[8], 'city': row[9]}
            listings.append(layout_dct)
    del listings[0] #remove header row  
    return listings

In [ ]:
# Function to create the dataframe, if I take that approach
def make_df(listings):
    df = pd.DataFrame(listings)
    times = (df.times.str.split('|', expand=True).rename(columns=lambda x: f"time_{x+1}"))
    times = times.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    times = times.apply(lambda x: x.str.replace('p', ' p.m.') if x.dtype == "object" else x)
    times = times.apply(lambda x: x.str.replace('a', ' a.m.') if x.dtype == "object" else x)
    times = times.apply(lambda x: x.str.replace(':00', '') if x.dtype == "object" else x)
    df = df.drop('times', axis=1)
    df = pd.concat([df, times], axis=1, join_axes=[df.index])
    df['streetaddy'] = df.addy.str.split(',').str[0]
    df.streetaddy = df.streetaddy.str.replace('Blvd', 'Blvd.')
    df.streetaddy = df.streetaddy.str.replace('Blvd..', 'Blvd.')
    df.streetaddy = df.streetaddy.str.replace('Avenue', 'Ave')
    df.streetaddy = df.streetaddy.str.replace('Ave', 'Ave.')
    df.streetaddy = df.streetaddy.str.replace('Ave..', 'Ave.')
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df['movieday'] = df['date'].dt.weekday_name
    df = df.drop('city', axis=1)
    df['cityaddy'] = df.addy.str.split(',').str[1]
    df = df.drop('addy', axis=1)
    return df 

In [14]:
# Define variables for each field in a row. Not sure if this is needed.
pn = []
for index, row in df.iterrows():
    form = df.format.to_string(header=False, index=False)
    stars = df.stars.to_string(header=False, index=False) 
    synopsis = df.synopsis.to_string(header=False, index=False)
    mpaa = df.mpaa.to_string(header=False, index=False)
    theater = df.theater.to_string(header=False, index=False) 
    title = df.title.to_string(header=False, index=False)
    time_1 = df.time_1.to_string(header=False, index=False)
    time_2 = df.time_2.to_string(header=False, index=False)
    time_3 = df.time_3.to_string(header=False, index=False)
    time_4 = df.time_4.to_string(header=False, index=False)
    time_5 = df.time_5.to_string(header=False, index=False)
    time_6 = df.time_6.to_string(header=False, index=False)
    time_7 = df.time_7.to_string(header=False, index=False)
    time_8 = df.time_8.to_string(header=False, index=False)
    streetaddy = df.streetaddy.to_string(header=False, index=False)
    movieday = df.movieday.to_string(header=False, index=False)
    cityaddy = df.cityaddy.to_string(header=False, index=False)
    pn += title
    pn += theater
    print(pn)

['R', 'a', 'l', 'p', 'h', ' ', 'B', 'r', 'e', 'a', 'k', 's', ' ', 't', 'h', 'e', ' ', 'I', 'n', 't', 'e', 'r', 'n', 'e', 't', '\n', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'B', 'o', 'h', 'e', 'm', 'i', 'a', 'n', ' ', 'R', 'h', 'a', 'p', 's', 'o', 'd', 'y', '\n', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'D', 'r', '.', ' ', 'S', 'e', 'u', 's', 's', "'", ' ', 'T', 'h', 'e', ' ', 'G', 'r', 'i', 'n', 'c', 'h', ' ', '(', '2', '0', '1', '8', ')', '\n', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'B', 'o', 'h', 'e', 'm', 'i', 'a', 'n', ' ', 'R', 'h', 'a', 'p', 's', 'o', 'd', 'y', '\n', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'V', 'e', 'n', 'o', 'm', ' ', '(', '2', '0', '1', 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
class Showing:
    def __init__(self, )
    
    
    
    
    
    class Cinema:
    def __init__(self, theater, street, city, market):
        self.theater = theater
        self.street = street
        self.city = city
        self.market = market
        
    def __str__(self):
        return f"{self.theater}, {self.stree}, {self.city}"
    
    def __repr__(self):
        return f"{self.theater}, {self.stree}, {self.city}"

class Movie:
    def __init__(self, title, mpaa, stars, synopsis):
        self.title = title
        self.mpaa = mpaa
        self.stars = stars
        self.synopsis = synopsis
        
    def __str__(sekf):
        return f"{title}, {mpaa}, {stars}, {synopsis}"
    
    def __str__(sekf):
        return f"{title}, {mpaa}, {stars}, {synopsis}"
    
class Showing:
    